# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
pip install openai panel python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install openai


In [3]:
import openai
print(openai.__version__)


2.7.1


In [4]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [5]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature,
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [25]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [28]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, of course, forsooth!


In [30]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Adrián'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Adrián! It's nice to meet you. How are you feeling today?


In [31]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to your personal information. You'll need to remind me of your name.


In [33]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Adrián'},
{'role':'assistant', 'content': "Hi Adri! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Adrián.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [14]:
!pip install jupyter_bokeh


  Obtaining dependency information for jupyter_bokeh from https://files.pythonhosted.org/packages/47/78/33b2294aad62e5f95b89a89379c5995c2bd978018387ef8bec79f6dc272c/jupyter_bokeh-4.0.5-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/148.6 kB ? eta -:--:--
   -- ------------------------------------- 10.2/148.6 kB ? eta -:--:--
   -- ------------------------------------- 10.2/148.6 kB ? eta -:--:--
   ---------------- ---------------------- 61.4/148.6 kB 656.4 kB/s eta 0:00:01
   ---------------------------------------- 148.6/148.6 kB 1.3 MB/s eta 0:00:00


In [15]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [17]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=1000),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'63ccb8e6-7601-4f01-af00-3a8a90fbf4db': {'version…

In [23]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "pizza": [
        {
            "item": "Pepperoni Pizza",
            "size": "Large",
            "price": 12.95
        },
        {
            "item": "Cheese Pizza",
            "size": "Large",
            "price": 10.95
        }
    ],
    "toppings": [],
    "drinks": [
        {
            "item": "Coke",
            "size": "Medium",
            "price": 2.00
        }
    ],
    "sides": [],
    "total price": 25.90
}
```


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise


 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative

1st version: vendor of cars

In [34]:
context = [
    {"role": "system", "content": """
You are AutoBot, a friendly AI assistant that helps customers choose and buy cars.

Your main goal is to understand the customer's preferences:
- budget,
- car type (SUV, sedan, sports, compact),
- **engine type (full-electric, hybrid, or combustion)**,
- color,
- and purpose (family, commuting, adventure, etc.).

Then recommend one or more cars from the catalog below.

🚗 **Car Catalog**
1. Tesla Model 3 — Full-Electric Sedan — €39,000  
2. Hyundai Ioniq 5 — Full-Electric SUV — €45,000  
3. Toyota Prius — Hybrid Hatchback — €29,000  
4. Kia Niro — Hybrid SUV — €33,000  
5. BMW X5 Hybrid — Plug-in Hybrid SUV — €65,000  
6. Ford Mustang — Combustion Sports — €48,000  
7. Toyota Corolla — Combustion Compact — €21,500  

💡 Rules:
- Always clarify whether the user wants **full-electric, hybrid, or combustion** before suggesting a model.
- If the user says “electric,” confirm whether they mean **100 % electric (no fuel)** or **hybrid (fuel + battery)**.
- When recommending, include the car type, engine type, and price.
- Be short, friendly, and helpful.
"""}
]


In [35]:
def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0.3):
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature
    )
    return response.choices[0].message.content


In [47]:
panels = []

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''  # clear input box

    # add user message to context
    context.append({'role': 'user', 'content': prompt})

    # get response from the model
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': response})

    # display conversation
    panels.append(pn.Row('👤 **You:**', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('🤖 **AutoBot:**', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)



In [ ]:
# DASHBOARD LAYOUT

inp = pn.widgets.TextInput(value="Hi!", placeholder="Ask about a car...")
button_conversation = pn.widgets.Button(name="Chat!", button_type="primary")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    pn.pane.Markdown("## 🚗 AutoBot — Your Car Sales Assistant", sizing_mode="stretch_width"),
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=1000)
)

dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'c73fbda7-b475-4d27-a07b-432b7a480e90': {'version…

2ND VERSION

In [49]:
text = f"""
In the lively streets of Seville, a young chef named María dreamed of creating the perfect paella. 
Every Sunday, she visited the market to choose fresh seafood, saffron, and vegetables. 
After months of practice, she finally mastered the timing and flavor balance, earning praise from her neighbors. 
Her dish became a local favorite, served with laughter, music, and the scent of orange blossoms in the air.
"""


In [50]:
prompt_1 = f"""
Follow these steps carefully:

1 - Summarize the text delimited by triple backticks in **one sentence**.
2 - Translate that summary into **Spanish**.
3 - Identify and list **all proper names** appearing in the Spanish summary.
4 - Output a **JSON object** with the following keys:
   - spanish_summary
   - num_names
   - names

Make sure to clearly separate each step’s output with line breaks.

Text:
```{text}```
"""


In [54]:
messages = [
    {"role": "user", "content": prompt_1}
]

response = get_completion_from_messages(messages)
print(response)



1 - María, a young chef in Seville, perfected her paella recipe and became a local favorite.

2 - María, una joven chef en Sevilla, perfeccionó su receta de paella y se convirtió en la favorita local.

3 - María

4 - 
{
   "spanish_summary": "María, una joven chef en Sevilla, perfeccionó su receta de paella y se convirtió en la favorita local.",
   "num_names": 1,
   "names": ["María"]
}


In [61]:
context = [
    {"role": "system", "content": """
You are a professional personal assistant specialized in helping with structured text tasks.
When the user asks for a recipe:
 - Choose a traditional or modern dish based on their input (e.g., cuisine, ingredient, or style).
 - Think step-by-step before answering.
 - Include: dish name, short description, list of ingredients (for 4 people), and numbered preparation steps.
 - Keep the language professional but friendly.
 - Optionally suggest a plating idea or a modern twist.
For all other tasks, help the user summarize, translate, or structure text clearly.
Always reason step-by-step before giving the final answer (Principle 2: Give the model time to think).
"""}
]


In [62]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0.5):
    """
    Sends a list of chat messages (system + user + assistant) to the model
    and returns the assistant's reply as text.
    """
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature  # controls creativity
    )

    # Extract the text content from the response
    return response.choices[0].message.content

In [64]:
#CHAT LOGIC

pn.extension()
panels = []  # To store all conversation exchanges

def collect_messages(_):
    prompt = inp.value_input  # Read user text
    inp.value = ''  # Clear the input box
    context.append({'role': 'user', 'content': prompt})  # Add user message to context

    # Get assistant response
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': response})

    # Display user + assistant messages
    panels.append(pn.Row('👤 **You:**', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('🤖 **Assistant:**', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [65]:
#  DASHBOARD UI (Your Layout)

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=1000),
)

dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'247fd7e8-de82-48ee-b550-ec871536a612': {'version…

Tactic 3: Ask the model to check whether conditions are satisfied

In [86]:
text=f"""
Deploying a logistic squadron requires careful preparation.
First, the commander must receive the movement order and confirm the mission requirements.
Next, coordinate with the operations and maintenance sections to ensure all vehicles are fully serviced and mission-ready.
Then, organize the convoy by assigning vehicle numbers, drivers, and loads according to the unit’s deployment plan.
Before departure, conduct a briefing to review the route, refueling points, communication procedures, and safety measures.
Once the convoy is ready, report departure to higher headquarters and maintain constant communication during movement.
Finally, confirm arrival at the destination and submit a logistic status report including personnel, fuel, and equipment availability.
"""


In [87]:
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of actions or procedures, rewrite those actions as a clear, numbered checklist in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

Keep each step short and action-oriented, using a professional military tone.
If the text does not contain a sequence of actions, write "No operational steps provided."

\"\"\"{text}\"\"\"
"""


In [94]:
def generate_operational_steps(text,  model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content 

In [95]:
response = generate_operational_steps(prompt, model="gpt-3.5-turbo", temperature=0)
print("Completion for Deployment Example:")
print(response)


Completion for Deployment Example:

Step 1 - Receive the movement order and confirm mission requirements.
Step 2 - Coordinate with operations and maintenance sections to ensure vehicles are fully serviced.
Step 3 - Organize the convoy by assigning vehicle numbers, drivers, and loads according to deployment plan.
Step 4 - Conduct a briefing to review route, refueling points, communication procedures, and safety measures.
Step 5 - Report departure to higher headquarters and maintain constant communication during movement.
Step 6 - Confirm arrival at destination and submit logistic status report.


- Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

 What I learned from this project is that the code crashes when the temperature is higher than 1.
I also learned the steps to build a chatbot: how to connect it with the OpenAI API, how to create the brain that processes messages, and how to design the dashboard for the user interface.
This helped me understand how all parts work together to make a functional chatbot.